In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path


TARGET_CRS = 'EPSG:2180'

In [2]:
def ensure_folder(rel_path: str):
    """
    Sprawdź i utwórz folder wskazany przez ścieżkę względną.
    """
    p = Path(rel_path)
    if p.exists():
        if p.is_dir():
            return True
        else:
            raise FileExistsError(f"Ścieżka istnieje i nie jest folderem: `{p}`")
    else:
        p.mkdir(parents=True, exist_ok=True)
        return True


def harmonization(teryt="no_teryt"):
    def nazwa_warstwy(file_path):
        split = file_path.split('_')
        return split[-2] + "_" + split[-1][0]


    def change_column_type(layers, layer_name, gdf):
        """
        Zmień typ kolumny, jeśli jest to konieczne
        """
        if 'CHANGE_TYPE' in layers[layer_name]:
            for col_name, target_type in layers[layer_name]['CHANGE_TYPE'].items():
                print(col_name, target_type)
                try:
                    if target_type == int or target_type == pd.Int64Dtype:
                        # Zrób maskę na wartości NaN, po czym non-NaN zmień na int     
                        if pd.api.types.is_numeric_dtype(gdf[col_name]):
                            mask = gdf[col_name].notna()
                            gdf.loc[mask, col_name] = gdf.loc[mask, col_name].astype(int)
                    elif target_type == str:
                        gdf[col_name] = gdf[col_name].astype(str)
                except TypeError as t:
                    print(f"❌ Błąd konwersji typu dla kolumny '{col_name}': {t}")
    
    
    warstwy = {
        # Mokradła
        'OIMK_A': {
            'COLUMNS': ['rodzaj', 'geometry'],
            'TARGET_COLUMNS': ['rodzaj', 'geometry']
        },
        # Szuwary
        'OISZ_A': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Wody powierzchniowe
        'PTWP_A': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Lasy
        'PTLZ_A': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Zabytki
        'KUZA_A': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Sport
        'KUSK_A': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Przemysł
        'KUPG_A': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Cmentarze
        'KUSC_A': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Miejscowości (PRG?)
        # Działki (PRG?)
        # Budynki
        'BUBD_A': {
            'COLUMNS': ['liczbaKondygnacji', 'informacjaDodatkowa', 'przewazajacaFunkcjaBudynku', 'geometry'],
            'TARGET_COLUMNS': ['liczba_kondygnacji', 'nazwa_budynku', 'funkcja_budynku', 'geometry'],
            'CHANGE_TYPE':
                {'liczba_kondygnacji': int}
        },
        # Strumienie (i rzeki)
        'SWRS_L': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Rowy melioracyjne
        'SWRM_L': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Drogi
        'SKJZ_L': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Tory kolejowe
        'SKTR_L': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Linie elektroenergetyczne
        'SULN_L': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Adresy (PRG?)
        # Stacje paliw (trzeba przekonwertować na punkty, tak wyglądają w Sycowie)
        'KUKO_A': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Stacje kolejowe i przystanki autobusowe
        'OIKM_P': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Transformatory
        'BUIT_P': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Wieże ciśnień, słupy energetyczne, wieże telekomunikacyjne, kominy
        'BUWT_P': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Studnia głębinowa
        'OIOR_P': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
        # Pomniki przyrody i drzewa
        'OIPR_P': {
            'COLUMNS': ['geometry'],
            'TARGET_COLUMNS': ['geometry']
        },
    }
    
    for key, value in warstwy.items():
        try:
            bdot_path = './dane_BDOT/PL.PZGiK.337.' + teryt[:4] + '__OT_' + key + '.xml'
            gdf_powiat = gpd.read_file(bdot_path)
            print(f"Wczytano {len(gdf_powiat)} obiektów z pliku {nazwa_warstwy(bdot_path)}")
            print(f"\n--- GeoDataFrame ({nazwa_warstwy(bdot_path)}) ---", gdf_powiat.head(3))
    
            # Ujednolicenie CRS
            if gdf_powiat.crs != TARGET_CRS:
                gdf_powiat = gdf_powiat.to_crs(TARGET_CRS)
                print("Reprojekcja danych BDOT10k zakończona.")
    
            # Iloczynowy spatial join i przycięcie do geometrii gminy Wińsko
            gdf_maska = gpd.GeoDataFrame(index=[0], crs=TARGET_CRS, geometry=[gmina.geometry])
            gdf_iloczyn = gpd.sjoin(gdf_powiat, gdf_maska, how="inner", predicate='intersects')
            gdf_iloczyn.drop(columns=['index_right'], inplace=True, errors='ignore')
            gdf_iloczyn = gpd.clip(gdf_iloczyn, gdf_maska)
    
            print(f"✅ Po wycięciu w granicach gminy {gmina.nazwa.capitalize()} pozostało {len(gdf_iloczyn)} obiektów.")
    
            # Sprawdzenie, czy dane zostały pomyślnie wczytane i wycięte, przystąpienie do harmonizacji
            if gdf_iloczyn is not None and not gdf_iloczyn.empty:
                layer_columns = warstwy[key]['COLUMNS']
                target_columns = warstwy[key]['TARGET_COLUMNS']
    
                gdf_harmonized = gpd.GeoDataFrame(columns=target_columns, crs=TARGET_CRS)
    
                # Wstępne przyporządkowanie
                for i in range(len(target_columns)):
                    gdf_harmonized[target_columns[i]] = gdf_iloczyn[layer_columns[i]]
                # Zmiana typu danych, jeśli potrzebna
                change_column_type(warstwy, key, gdf_harmonized)
    
                print(f"\n--- Zharmonizowana GeoDataFrame ({key}) ---")
                print(gdf_harmonized.head(2))
                output_filename = 'bdot10k_' + key + '_harmonized.gpkg'
                
                if ensure_folder('./dane_Harmonizacja'):
                    if ensure_folder('./dane_Harmonizacja/' + teryt):
                        gdf_harmonized.to_file(r'./dane_Harmonizacja/' + teryt + '/' + output_filename, driver='GPKG')
                        print(f"✅ Zharmonizowane dane zapisane do pliku: {output_filename}")
    
            elif gdf_iloczyn is not None and gdf_iloczyn.empty:
                print(f"\n⚠️ Brak obiektów BDOT10k w pliku {key}, które by przecinały geometrię gminy {gmina.nazwa}.")
        except FileNotFoundError:
            print(f"❌ Błąd: Plik **{bdot_path}** nie został znaleziony! Upewnij się, że pobrałeś paczkę i zmieniłeś ścieżkę.")
        except Exception as e:
            print(f"❌ Wystąpił błąd podczas przetwarzania: {e}")

In [3]:
gmina_path = 'GMN.shp' # plik SHP z geometrią gminy (tutaj brany jest pierwszy rekord)
gmina = None

try:
    gdf_teryt = gpd.read_file(gmina_path, encoding='utf-8')
    print(f"Wczytano {len(gdf_teryt)} obiektów gmin.")
    
    # Ujednolicenie CRS na EPSG 2180
    if gdf_teryt.crs != 'EPSG:2180':
        gdf_teryt = gdf_teryt.to_crs('EPSG:2180')
        print(f"Reprojekcja do EPSG:2180 zakończona.")

    # Znalezienie odpowiednich kolumn dla nazwy i typu
    kolumna_gmin = next((col for col in gdf_teryt.columns if 'NAZWA' in col.upper()), None)
    
    if kolumna_gmin:
        gdf_gmina = gdf_teryt[gdf_teryt[kolumna_gmin].notna()]
        gmina = gdf_gmina.iloc[0]
        print(gdf_gmina)
        for row in gdf_gmina.itertuples():
            teryt_val = getattr(row, 'teryt', None)
            if teryt_val is None or type(teryt_val) != str:
                continue
            print(f"✅ Znaleziono gminę {gmina.nazwa} (CRS: {gdf_gmina.crs}).")
            harmonization(teryt_val)
    else:
        raise ValueError("❌ Nie znaleziono kolumny zawierającej nazwę gminy. Sprawdź `gdf_teryt.head()`.")
except FileNotFoundError:
    print(f"❌ Błąd: Plik '{gmina_path}' nie został znaleziony! Upewnij się, że pobrałeś plik SHP z granicami gmin (PRG) i umieściłeś go w odpowiednim miejscu.")
except Exception as e:
    print(f"❌ Wystąpił błąd ogólny podczas przetwarzania TERYT: {e}")

Wczytano 1 obiektów gmin.
   fid   nazwa     teryt                                           geometry
0  1.0  Wińsko  022202_2  POLYGON ((346021.83 402656.06, 346011.47 40264...
✅ Znaleziono gminę Wińsko (CRS: EPSG:2180).
Wczytano 530 obiektów z pliku OIMK_A

--- GeoDataFrame (OIMK_A) ---                                               gml_id  \
0  PL.PZGiK.337.BDOT10k_6702a262-692f-11ef-9079-b...   
1  PL.PZGiK.337.BDOT10k_66fde786-692f-11ef-a47c-b...   
2  PL.PZGiK.337.BDOT10k_66f44aaa-692f-11ef-a02b-b...   

                              lokalnyId        przestrzenNazw  \
0  6702a262-692f-11ef-9079-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   
1  66fde786-692f-11ef-a47c-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   
2  66f44aaa-692f-11ef-a02b-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   

                wersja poczatekWersjiObiektu oznaczenieZmiany  \
0  2024-09-02T12:00:00   2024-09-02T12:00:00     MGW/105/2024   
1  2024-09-02T12:00:00   2024-09-02T12:00:00     MGW/105/2024   
2  2024-09-02T12:00:00   2024

Skipping field funkcjaSzczegolowaBudynku: unsupported OGR type: 5


✅ Zharmonizowane dane zapisane do pliku: bdot10k_KUPG_A_harmonized.gpkg
Wczytano 50 obiektów z pliku KUSC_A

--- GeoDataFrame (KUSC_A) ---                                               gml_id  \
0  PL.PZGiK.337.BDOT10k_3e386f7c-692f-11ef-896c-b...   
1  PL.PZGiK.337.BDOT10k_3e35fe56-692f-11ef-a1d5-b...   
2  PL.PZGiK.337.BDOT10k_3e386f48-692f-11ef-a642-b...   

                              lokalnyId        przestrzenNazw  \
0  3e386f7c-692f-11ef-896c-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   
1  3e35fe56-692f-11ef-a1d5-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   
2  3e386f48-692f-11ef-a642-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   

                wersja poczatekWersjiObiektu oznaczenieZmiany  \
0  2024-09-02T12:00:00   2024-09-02T12:00:00     MGW/105/2024   
1  2024-09-02T12:00:00   2024-09-02T12:00:00     MGW/105/2024   
2  2024-09-02T12:00:00   2024-09-02T12:00:00     MGW/105/2024   

  zrodloDanychGeometrycznych kategoriaIstnienia       informacjaDodatkowa  \
0               ortofotomapa      eks

Skipping field numerDrogi: unsupported OGR type: 5


✅ Po wycięciu w granicach gminy Wińsko pozostało 1023 obiektów.

--- Zharmonizowana GeoDataFrame (SWRM_L) ---
                                               geometry
2785  LINESTRING (331750.141 392375.516, 331755.23 3...
1503  LINESTRING (331803.897 392502.823, 331798.2 39...
✅ Zharmonizowane dane zapisane do pliku: bdot10k_SWRM_L_harmonized.gpkg
Wczytano 14982 obiektów z pliku SKJZ_L

--- GeoDataFrame (SKJZ_L) ---                                               gml_id  \
0  PL.PZGiK.337.BDOT10k_706e53d6-6930-11ef-86f4-b...   
1  PL.PZGiK.337.BDOT10k_70b98da9-6930-11ef-b457-b...   
2  PL.PZGiK.337.BDOT10k_70dcf419-6930-11ef-84a5-b...   

                              lokalnyId        przestrzenNazw  \
0  706e53d6-6930-11ef-86f4-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   
1  70b98da9-6930-11ef-b457-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   
2  70dcf419-6930-11ef-84a5-b8ca3aa2e5c2  PL.PZGiK.337.BDOT10k   

                wersja poczatekWersjiObiektu oznaczenieZmiany  \
0  2024-09-02T12:00:00   2024